# MySpotify

## Install requirements

In [ ]:
%pip install -r requirements.txt

## Initilizing MySpotify Module

In [1]:
%reload_ext autoreload
%autoreload 2

DEBUG_MODE = True

from MySpotify import MySpotify, os
import requests

# Need always to reload the link to download the data
if not os.path.exists("data.zip"):
    download_link = ""

    response = requests.get(download_link)
    with open("data.zip", "wb") as file:
        file.write(response.content)


SpotifyModule = MySpotify("data.zip", DEBUG_MODE)

All files are present!


/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocessing Data

### Convert files from csv to parquet

In [ ]:
SpotifyModule.convert_files()

### Merge the Data into one parquet file

In [ ]:
SpotifyModule.PreProcess_Data()

## Display DataFrame

In [ ]:
from IPython.display import HTML, display, clear_output

def display_scrollable_df(df, height=400):
    html = df.to_html()
    scrollable = f'<div style="height: {height}px; overflow: auto;">{html}</div>'
    display(HTML(scrollable))

## Top Tracks

### Top 100 tracks by the play count

In [ ]:
TopTracksNum = 100
best_tracks = SpotifyModule.get_Top_Tracks(TopTracksNum)
display_scrollable_df(best_tracks)

### Top 100 tracks by genre

In [ ]:

TopTracksByGenreNum = 100
genre_list = ["Rock", "Rap", "Jazz", "Electronic", "Pop", "Blues", "Country", "Reggae", "New Age"]
for genre in genre_list:
    clear_output(wait=True)
    best_tracks_by_genre = SpotifyModule.Get_TopTracks_By_Genre(TopTracksByGenreNum, genre)
    display_scrollable_df(best_tracks_by_genre)
    key = input("Press Enter to continue to the next genre, or type something and press Enter to exit: ")
    if key != "":
        break


## Collections

### Using Baseline

In [ ]:
Collections = ["love", "war","happiness", "loneliness", "money"]
Num_of_tracks = 50
for theme in Collections:
    clear_output(wait=True)
    Baseline = SpotifyModule.Baseline(theme, Num_of_tracks)
    display_scrollable_df(Baseline)
    key = input("Press Enter to continue to the next genre, or type something and press Enter to exit: ")
    if key != "":
        break

### Using Word2Vec

In [ ]:
Collections = ["love", "war","happiness", "loneliness", "money"]
Num_of_tracks = 50
for theme in Collections:
    clear_output(wait=True)
    Word2Vec = SpotifyModule.Word2Vec(theme, Num_of_tracks)
    display_scrollable_df(Word2Vec)
    key = input("Press Enter to continue to the next genre, or type something and press Enter to exit: ")
    if key != "":
        break

### Using Classifier

In [ ]:
Collections = ["love", "war","happiness", "loneliness", "money"]
Num_of_tracks = 50
for theme in Collections:
    clear_output(wait=True)
    Classification = SpotifyModule.Classification(Collections, theme, Num_of_tracks)
    display_scrollable_df(Classification)
    key = input("Press Enter to continue to the next genre, or type something and press Enter to exit: ")
    if key != "":
        break


## Recommendations

### Similar to a User listening

In [2]:
user_id = "19a63dbfea18904260bfe9c35cabb711ce01d2a6"
SpotifyModule._Recommendations.get_similar_tracks(user_id)

Mapping Songs: 100%|██████████| 384546/384546 [00:00<00:00, 1995305.03it/s]


Fitting Model


100%|██████████| 1/1 [00:08<00:00,  8.20s/it]


2


ValueError: too many values to unpack (expected 2)

In [ ]:
from pyarrow.parquet import ParquetFile
import pandas as pd

pf = ParquetFile("data/parquet/Merged_tracks_data.parquet")

# df : pd.DataFrame = pf.read().to_pandas()
print(pf.metadata.schema.names)